# Particionado vertical

El particionado vertical nos puede ayudar cuando contamos con tablas muy anchas (Con columnas muy amplias como por ejemplo VARCHAR(MAX), VARBINARY o IMAGE) y dichas columnas no son utilizadas en la mayoría de consultas. Al mandarlas a otra partición podemos acelerar aquellas consultas que no ocupen dichas columnas y cuyo plan de ejecución contenga el operador de CLUSTERED INDEX SCAN.

**1\. Setup**

Vamos a crear una tabla con una columna de tipo IMAGE, el objetivo de esta práctica será aislar esa columna para mejorar el rendimiento

In [1]:
CREATE DATABASE PartitionTestDB
GO
USE PartitionTestDB
GO

CREATE TABLE Employee(
	EmployeeID INT primary key identity,
	FirstName VARCHAR(128),
	LastName VARCHAR(128),
	Picture IMAGE
);

INSERT INTO Employee VALUES ('Juan','Ramirez',(SELECT * FROM OPENROWSET(BULK N'C:\img\1.png', SINGLE_BLOB) AS T1))
INSERT INTO Employee VALUES ('Jose','Perez',(SELECT * FROM OPENROWSET(BULK N'C:\img\1.png', SINGLE_BLOB) AS T1))
INSERT INTO Employee VALUES ('Ana','Lopez',(SELECT * FROM OPENROWSET(BULK N'C:\img\2.jpg', SINGLE_BLOB) AS T1))


Commands completed successfully.

Commands completed successfully.

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.812

**2\. Extracción de las columnas EmployeeID y Picture en una tabla temporal**

In [2]:
SELECT EmployeeID, Picture INTO #EmployeePictureTemp FROM Employee

(3 rows affected)

Total execution time: 00:00:00.085

**3\. Creación de la nueva tabla para la partición**

In [4]:
CREATE TABLE EmployeePicture (
    EmployeeID INT PRIMARY KEY,
    Picture IMAGE,

    CONSTRAINT EmployeePicture_FK FOREIGN KEY (EmployeeID) REFERENCES Employee(EmployeeID)
);

Commands completed successfully.

Total execution time: 00:00:00.004

**4\. Traslado de la información de la tabla temporal hacia la nueva partición y borrado de columna Picture**

In [5]:
INSERT INTO EmployeePicture(EmployeeID, Picture)
SELECT T.EmployeeID, T.Picture
FROM #EmployeePictureTemp AS T

(3 rows affected)

Total execution time: 00:00:00.006

In [6]:
ALTER TABLE Employee DROP COLUMN Picture

Commands completed successfully.

Total execution time: 00:00:00.010

**5\. Consultas finales**

In [7]:
SELECT * FROM Employee;

(3 rows affected)

Total execution time: 00:00:00.056

EmployeeID,FirstName,LastName
1,Juan,Ramirez
2,Jose,Perez
3,Ana,Lopez


In [8]:
SELECT * FROM EmployeePicture;

(3 rows affected)

Total execution time: 00:00:00.019

EmployeeID,Picture
1,...
2,...
3,...
